In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jahanzebnaeem/wlpsl

Dataset URL: https://www.kaggle.com/datasets/jahanzebnaeem/wlpsl
License(s): other
100% 350M/350M [00:18<00:00, 19.9MB/s]
100% 350M/350M [00:18<00:00, 19.7MB/s]


In [ ]:
from zipfile import ZipFile
data='/content/wlpsl.zip'
with ZipFile(data,'r') as zip:
  zip.extractall()
  print("Data set is extracted")

Data set is extracted


In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_face_mesh = mp.solutions.face_mesh

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
import os
import shutil

# Source folder path
source_folder = "/content/WLPSL/Videos"

# Destination folder path
destination_folder = "/content/WLPSL/Videos1"

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# List of subfolders or files to include
include = ["alert", "careful", "cheap"]

# Iterate through the source folder
for item in os.listdir(source_folder):
    source_path = os.path.join(source_folder, item)
    destination_path = os.path.join(destination_folder, item)

    # Check if the item is in the include list
    if item in include:
        # Copy folder or file
        if os.path.isdir(source_path):
            shutil.copytree(source_path, destination_path)
        else:
            shutil.copy2(source_path, destination_path)

print(f"Selected items copied to {destination_folder}")


Selected items copied to /content/WLPSL/Videos1


In [ ]:
import shutil
shutil.rmtree("/content/WLPSL/Videos1")

In [ ]:
import cv2
import os
import numpy as np

# Directory containing videos
video_folder = '/content/WLPSL/Videos1'  # Update with the path to your videos
# Directory to save extracted frames
output_folder = 'MP_Data'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
# Number of frames to extract per video
frames_per_sequence = 30

# Function to extract exactly 30 frames uniformly
def extract_frames_from_video(video_path, output_folder, gesture_folder, video_index):
  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    video_capture = cv2.VideoCapture(video_path)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame indices to sample
    frame_indices = np.linspace(0, total_frames - 1, frames_per_sequence, dtype=int)

    extracted_count = 0
    output_subfolder = os.path.join(output_folder, gesture_folder, str(video_index))
    os.makedirs(output_subfolder, exist_ok=True)

    frame_count = 0
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)


        if frame_count in frame_indices:
            # Preprocess frame (example: using Mediapipe or other processing)
            keypoints = extract_keypoints(results)  # Replace with your desired preprocessing logic
            frame_filename = os.path.join(output_subfolder, f"{extracted_count + 1}")
            np.save(frame_filename, keypoints)
            extracted_count += 1

        frame_count += 1

    video_capture.release()

# Loop through all videos in the folder
for gesture_folder in os.listdir(video_folder):
    gesture_path = os.path.join(video_folder, gesture_folder)

    if os.path.isdir(gesture_path):  # Check if it's a folder for gestures
        video_index = 1  # Start numbering videos from 1
        for filename in os.listdir(gesture_path):
            if filename.endswith(".mp4") or filename.endswith(".avi"):  # Adjust for your video file type
                video_path = os.path.join(gesture_path, filename)
                extract_frames_from_video(video_path, output_folder, gesture_folder, video_index)
                print(f"Frames extracted for {filename} in gesture folder '{gesture_folder}'")
                video_index += 1


Frames extracted for 4.mp4 in gesture folder 'ready'
Frames extracted for 6.mp4 in gesture folder 'ready'
Frames extracted for 1.mp4 in gesture folder 'ready'
Frames extracted for 8.mp4 in gesture folder 'ready'
Frames extracted for 5.mp4 in gesture folder 'ready'
Frames extracted for 7.mp4 in gesture folder 'ready'
Frames extracted for 3.mp4 in gesture folder 'ready'
Frames extracted for 2.mp4 in gesture folder 'ready'
Frames extracted for 4.mp4 in gesture folder 'good'
Frames extracted for 6.mp4 in gesture folder 'good'
Frames extracted for 1.mp4 in gesture folder 'good'
Frames extracted for 8.mp4 in gesture folder 'good'
Frames extracted for 5.mp4 in gesture folder 'good'
Frames extracted for 7.mp4 in gesture folder 'good'
Frames extracted for 3.mp4 in gesture folder 'good'
Frames extracted for 2.mp4 in gesture folder 'good'
Frames extracted for 4.mp4 in gesture folder 'dumb'
Frames extracted for 6.mp4 in gesture folder 'dumb'
Frames extracted for 1.mp4 in gesture folder 'dumb'
Fram

In [ ]:

actions=[action_name for action_name in os.listdir("/content/MP_Data")]

In [ ]:
len(actions)

3

In [ ]:
actions=np.array(actions)

In [ ]:
actions

array(['ready', 'good', 'dumb'], dtype='<U5')

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
len(label_map)

3

In [ ]:
label_map

{'ready': 0, 'good': 1, 'dumb': 2}

In [ ]:
sequence_length=30
DATA_PATH="/content/MP_Data"
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
          frame_num=frame_num+1
          res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
          window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

(24, 30, 1662)

In [ ]:
np.array(labels).shape

(24,)

In [ ]:
X = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=0.05)

NameError: name 'X' is not defined

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=150, callbacks=[tb_callback])

Epoch 1/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - categorical_accuracy: 0.3182 - loss: 1.0985
Epoch 2/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - categorical_accuracy: 0.3182 - loss: 1.1013
Epoch 3/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - categorical_accuracy: 0.4091 - loss: 1.0903
Epoch 4/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - categorical_accuracy: 0.2727 - loss: 5.6408
Epoch 5/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - categorical_accuracy: 0.3182 - loss: 1.1200
Epoch 6/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - categorical_accuracy: 0.3636 - loss: 1.1196
Epoch 7/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - categorical_accuracy: 0.3636 - loss: 1.0666
Epoch 8/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - categorical_accuracy: 0.4091 - loss: 1.9218
Epoch 9/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - categorical_accuracy: 0.3182 - loss: 1.6827
Epoch 10/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - categorical_accuracy: 0.2727 - loss: 3.6936
Epoch 11/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [ ]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print test accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Test Accuracy: 0.00%


In [ ]:
model.save('action.h5')

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
from google.colab import files

In [ ]:
files.download('/content/MP_Data')

FileNotFoundError: Cannot find file: /content/MP_Data

try with lstm and cnn

In [ ]:
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle

In [ ]:

# Parameters
FRAME_SIZE = (64, 64)      # Resize frames to 64x64
MAX_FRAMES = 30            # Max number of frames per video
DATA_DIR = "/content/WLPSL/Videos"  # Update with your dataset path

data = []
labels = []

# Loop through dataset directories
for category in os.listdir(DATA_DIR):
    category_path = os.path.join(DATA_DIR, category)
    if not os.path.isdir(category_path):
        continue

    for video_file in os.listdir(category_path):
        video_path = os.path.join(category_path, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, FRAME_SIZE)
            frame = frame / 255.0  # Normalize
            frames.append(frame)

        cap.release()

        # Pad or truncate frames to MAX_FRAMES
        frames = pad_sequences([frames], maxlen=MAX_FRAMES, padding='post', dtype='float32')[0]

        data.append(frames)
        labels.append(category)

# Convert to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Encode labels
le = LabelEncoder()
labels = le.fit_transform(labels)

# Save the preprocessed data
with open("hand_sign_data.pkl", "wb") as f:
    pickle.dump((data, labels), f)


In [ ]:
from sklearn.model_selection import train_test_split

# Load data
with open("hand_sign_data.pkl", "rb") as f:
    data, labels = pickle.load(f)

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, LSTM, Dense, Flatten, Dropout

model = Sequential()

# Feature extraction with CNN
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(MAX_FRAMES, *FRAME_SIZE, 3)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

# LSTM for temporal processing
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(len(np.unique(labels)), activation='softmax'))




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#model.summary()

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=16)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 398ms/step - accuracy: 0.0264 - loss: 3.6792 - val_accuracy: 0.0000e+00 - val_loss: 3.5702
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 204ms/step - accuracy: 0.0394 - loss: 3.4450 - val_accuracy: 0.0000e+00 - val_loss: 3.5011
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 204ms/step - accuracy: 0.0470 - loss: 3.4336 - val_accuracy: 0.0400 - val_loss: 3.4699
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 205ms/step - accuracy: 0.0458 - loss: 3.4362 - val_accuracy: 0.0000e+00 - val_loss: 3.4665
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - accuracy: 0.0646 - loss: 3.4282 - val_accuracy: 0.0000e+00 - val_loss: 3.4819
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 204ms/step - accuracy: 0.0498 - loss: 3.4284 - val_accuracy: 0.0000e+00 - val_loss: 3.4833
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 205ms/step - accuracy: 0.0655 - loss: 3.4384 - val_accuracy: 0.0000e+00 - val_loss: 3.4691
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 208ms/step - accuracy: 0.0912 - loss: 

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.0000e+00 - loss: 3.4861
Test Accuracy: 0.00%


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dropout, Dense

# Load MobileNetV2 for feature extraction
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(*FRAME_SIZE, 3))

model = Sequential()

# TimeDistributed wrapper for applying the CNN to each frame
model.add(TimeDistributed(base_model, input_shape=(MAX_FRAMES, *FRAME_SIZE, 3)))
model.add(TimeDistributed(Flatten()))

# LSTM layer for temporal sequence processing
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))

# Dense output layer
model.add(Dense(len(np.unique(labels)), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


<ipython-input-12-8e2ea42915a5>:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(*FRAME_SIZE, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=3)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.1893 - loss: 2.9795 - val_accuracy: 0.0400 - val_loss: 3.2494 - learning_rate: 0.0010
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.0531 - loss: 3.1488 - val_accuracy: 0.0800 - val_loss: 3.2293 - learning_rate: 0.0010
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.1510 - loss: 2.9563 - val_accuracy: 0.0400 - val_loss: 3.5467 - learning_rate: 0.0010
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.1148 - loss: 3.0845 - val_accuracy: 0.0000e+00 - val_loss: 3.3890 - learning_rate: 0.0010
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.1374 - loss: 3.0000 - val_accuracy: 0.0000e+00 - val_loss: 3.3543 - learning_rate: 0.0010
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.1146 - loss: 3.1005 - val_accuracy: 0.0400 - val_loss: 3.2993 - learning_rate: 5.0000e-04
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.1046 - loss: 2.9959 - va

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = dict(enumerate(class_weights))

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=16, class_weight=class_weights_dict)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 275s 1s/step - accuracy: 0.0988 - loss: 2.9684 - val_accuracy: 0.0400 - val_loss: 3.3198
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.1047 - loss: 2.9456 - val_accuracy: 0.0000e+00 - val_loss: 3.3492
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.1415 - loss: 2.8747 - val_accuracy: 0.0400 - val_loss: 3.3488
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.1308 - loss: 2.8962 - val_accuracy: 0.0000e+00 - val_loss: 3.4701
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.1587 - loss: 2.9391 - val_accuracy: 0.0400 - val_loss: 3.4955
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.1340 - loss: 2.8463 - val_accuracy: 0.0400 - val_loss: 3.4452
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.1669 - loss: 2.7996 - val_accuracy: 0.0400 - val_loss: 3.5642
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.1896 - loss: 2.7233 - val_accuracy: 0.0400 - 

In [ ]:
import cv2
import os
import numpy as np

# Directory containing videos
video_folder = '/content/WLPSL/Videos1'  # Update with the path to your videos
# Directory to save extracted frames
output_folder = 'MP_Data'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
# Number of frames to extract per video
frames_per_sequence = 30

# Function to extract exactly 30 frames uniformly
def extract_frames_from_video(video_path, output_folder, gesture_folder, video_index):
  #with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    video_capture = cv2.VideoCapture(video_path)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame indices to sample
    frame_indices = np.linspace(0, total_frames - 1, frames_per_sequence, dtype=int)

    extracted_count = 0
    output_subfolder = os.path.join(output_folder, gesture_folder, str(video_index))
    os.makedirs(output_subfolder, exist_ok=True)

    frame_count = 0
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        #image, results = mediapipe_detection(frame, holistic)
        #draw_styled_landmarks(image, results)


        if frame_count in frame_indices:
            # Preprocess frame (example: using Mediapipe or other processing)
            #keypoints = extract_keypoints(results)  # Replace with your desired preprocessing logic
            frame_filename = os.path.join(output_subfolder, f"{extracted_count + 1}.jpg")
            cv2.imwrite(frame_filename, frame)
            extracted_count += 1

        frame_count += 1

    video_capture.release()

# Loop through all videos in the folder
for gesture_folder in os.listdir(video_folder):
    gesture_path = os.path.join(video_folder, gesture_folder)

    if os.path.isdir(gesture_path):  # Check if it's a folder for gestures
        video_index = 1  # Start numbering videos from 1
        for filename in os.listdir(gesture_path):
            if filename.endswith(".mp4") or filename.endswith(".avi"):  # Adjust for your video file type
                video_path = os.path.join(gesture_path, filename)
                extract_frames_from_video(video_path, output_folder, gesture_folder, video_index)
                print(f"Frames extracted for {filename} in gesture folder '{gesture_folder}'")
                video_index += 1


Frames extracted for 6.mp4 in gesture folder 'cheap'
Frames extracted for 7.mp4 in gesture folder 'cheap'
Frames extracted for 2.mp4 in gesture folder 'cheap'
Frames extracted for 8.mp4 in gesture folder 'cheap'
Frames extracted for 1.mp4 in gesture folder 'cheap'
Frames extracted for 4.mp4 in gesture folder 'cheap'
Frames extracted for 3.mp4 in gesture folder 'cheap'
Frames extracted for 5.mp4 in gesture folder 'cheap'
Frames extracted for 6.mp4 in gesture folder 'alert'
Frames extracted for 7.mp4 in gesture folder 'alert'
Frames extracted for 2.mp4 in gesture folder 'alert'
Frames extracted for 8.mp4 in gesture folder 'alert'
Frames extracted for 1.mp4 in gesture folder 'alert'
Frames extracted for 4.mp4 in gesture folder 'alert'
Frames extracted for 3.mp4 in gesture folder 'alert'
Frames extracted for 5.mp4 in gesture folder 'alert'
Frames extracted for 6.mp4 in gesture folder 'careful'
Frames extracted for 7.mp4 in gesture folder 'careful'
Frames extracted for 2.mp4 in gesture fold

In [ ]:
import shutil
shutil.rmtree("/content/MP_Data1")

In [ ]:
import shutil

# Folder path to compress
folder_path = '/content/MP_Data/alert/8'
# Output zip file
zip_file_path = '/content/alert_8.zip'

# Compress the folder into a zip file
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)


'/content/alert_8.zip'

In [ ]:
from google.colab import files

# Download the zip file
files.download(zip_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!kaggle datasets download -d saadbutt321/pakistan-sign-language-dataset

Dataset URL: https://www.kaggle.com/datasets/saadbutt321/pakistan-sign-language-dataset
License(s): GPL-2.0
  0% 0.00/4.09M [00:00<?, ?B/s]
100% 4.09M/4.09M [00:00<00:00, 64.2MB/s]


In [ ]:
from zipfile import ZipFile
data='/content/pakistan-sign-language-dataset.zip'
with ZipFile(data,'r') as zip:
  zip.extractall()
  print("Data set is extracted")

Data set is extracted
